# Table of Content

<a name="outline"></a>

## Setup

- [A](#seca) External Imports
- [B](#secb) Internal Imports
- [C](#secc) Configurations and Paths 
- [D](#secd) JAX Interface
- [E](#sece) General Utility Functions


## Clustering

- [1](#sec2) Disease Embeddings Clustering
- [2](#sec3) Subject Embeddings Clustering

<a name="seca"></a>

### A External Imports [^](#outline)

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

<a name="secb"></a>

### B Internal Imports [^](#outline)

In [2]:


sys.path.append("..")

from lib import utils as U
from lib.ehr.dataset import load_dataset

# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_FILE = f'{HOME}/GP/ehr-data/cprd-data/DUMMY_DATA.csv'
SOURCE_DIR = os.path.abspath("..")

<a name="secc"></a>

### C Configurations and Paths [^](#outline)

In [3]:
output_dir = 'cprd_artefacts'
Path(output_dir).mkdir(parents=True, exist_ok=True)


In [4]:
with U.modified_environ(DATA_FILE=DATA_FILE):
    cprd_dataset = load_dataset('CPRD')
   

In [5]:
output_dir = 'cprd_clustering_artefacts'
Path(output_dir).mkdir(parents=True, exist_ok=True)


<a name="secd"></a>

### D JAX Interface [^](#outline)

### Configuration should match the same configuration used in training in `cprd2_dx_training.ipynb`

In [6]:

%load_ext autoreload
%autoreload 2

from lib.ehr.coding_scheme import DxLTC212FlatCodes, DxLTC9809FlatMedcodes, EthCPRD5, EthCPRD16
from lib.ehr import OutcomeExtractor, FirstOccurrenceOutcomeExtractor
from lib.ehr import Subject_JAX
from lib.ehr import StaticInfoFlags

%load_ext autoreload
%autoreload 2

code_scheme = {
    'dx': DxLTC9809FlatMedcodes(), # other options 
    'outcome': OutcomeExtractor('dx_cprd_ltc9809'),
    # Comment above^, and uncomment below, to consider only the first occurrence of codes per subject.
    # 'outcome': FirstOccurrenceOutcomeExtractor('dx_cprd_ltc9809'),
    'eth': EthCPRD5()
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

static_info_flags = StaticInfoFlags(
 gender=True,
 age=True,
 idx_deprivation=True,
 ethnicity=EthCPRD5(), # <- include it by the category of interest, not just 'True'.
)
cprd_interface = Subject_JAX.from_dataset(cprd_dataset, code_scheme=code_scheme, static_info_flags=static_info_flags)
cprd_all_subjects = list(cprd_interface.keys())
cprd_splits = cprd_interface.random_splits(split1=0.7, split2=0.85, random_seed=42)

<a name="sec1"></a>

## 1 Disease Embeddings Clustering on CPRD [^](#outline)

In [8]:


# Should be the same one used in JAX interface in the training notebook.
dx_scheme = DxLTC9809FlatMedcodes()


In [9]:
# scheme indices (textual code -> integer index)
dx_scheme.index

# reverse index (integer index -> textual code)
idx2code = {idx: code for code, idx in dx_scheme.index.items()}

### 1.A GloVe Based Disease Embeddings

Get the coocurrence matrix

In [15]:
cprd_cooc = cprd_interface.dx_coocurrence(cprd_all_subjects, window_size_days=365)

from lib.embeddings import train_glove

cprd_glove = train_glove(cprd_cooc, embeddings_size=100, iterations=500, prng_seed=0)

cprd_glove

array([[-0.00060291,  0.00388628,  0.0011434 , ..., -0.00505025,
         0.00322211, -0.00227745],
       [ 0.00287454, -0.00062646,  0.00173732, ..., -0.00019625,
        -0.00357063,  0.0035935 ],
       [ 0.00176105,  0.00265408, -0.00154581, ...,  0.00362902,
        -0.00319561, -0.00013699],
       ...,
       [-0.00794647, -0.00551591,  0.00785181, ..., -0.00017621,
         0.00284466,  0.00258541],
       [-0.00094812,  0.00257151,  0.00391616, ...,  0.00557593,
         0.00580394, -0.00319773],
       [ 0.00202193,  0.00555665, -0.00156006, ...,  0.00220455,
        -0.00023156,  0.00452634]])

### 1.B Predictor Based Disease Embeddings

TODO

### 1.C Predictor Based Subject Embeddings

TODO

In [32]:


# def embeddings_dictionary(clf):
#     model, state = cprd_predictors[clf]
#     params = model.get_params(state)
#     # Embeddings Mat
#     dx_G = model.dx_emb.compute_embeddings_mat(params['dx_emb'])

#     embeddings_dict = {}
#     for code, idx in dx_scheme.index.items():
#         in_vec = np.zeros((cprd_interface.dx_dim, ))
#         in_vec[idx] = 1.
#         out_vec = model.dx_emb.encode(dx_G, in_vec)
#         embeddings_dict[code] = out_vec
#     return embeddings_dict

# icenode_emb = embeddings_dictionary('ICE-NODE')
# icenode_uni_emb = embeddings_dictionary('ICE-NODE_UNIFORM')
# retain_emb = embeddings_dictionary('RETAIN')
# gru_emb = embeddings_dictionary('GRU')


# def subject_embeddings_dictionary(clf):
#     model, state = cprd_predictors[clf]
#     # All subjects in the study are passed
#     return model.subject_embeddings(state, cprd_interface.subjects)

# icenode_subj_emb = subject_embeddings_dictionary('ICE-NODE')
# icenode_subj_uni_emb = subject_embeddings_dictionary('ICE-NODE_UNIFORM')
# retain_subj_emb = subject_embeddings_dictionary('RETAIN')
# gru_subj_emb = subject_embeddings_dictionary('GRU')

/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/haiku/_src/data_structures.py:145: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)
